In [1]:
# ============================================================
# 14번: AutoGluon + KNN Imputation + 12번 피처셋
# - 12번과 동일한 imputation + 피처 엔지니어링
# - AutoGluon이 자체 앙상블 (LGB/CAT/XGB/RF/NN 등)
# ============================================================

import os, gc, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.impute import KNNImputer
from autogluon.tabular import TabularPredictor

# -------------------------
# Config
# -------------------------
DATA_DIR = "../data"
OUT_DIR  = "../outputs"
os.makedirs(OUT_DIR, exist_ok=True)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"
N_FOLDS = 5
SEED = 42

# AutoGluon 설정
AG_TIME_LIMIT = 7200        # 1시간 (fold당). 늘리면 성능↑
AG_PRESETS = "best_quality"  # best_quality > high_quality > medium_quality
AG_NUM_STACK = 1             # stacking 레벨
AG_NUM_BAG = 8               # bagging fold 수


# ===================================================================
# 1) Imputation (12번과 동일)
# ===================================================================
def impute_missing(train_df, test_df):
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    di_fill_cols = [
        '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '총 생성 배아 수', '이식된 배아 수',
        '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수',
        '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수',
        '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수',
        '난자 채취 경과일', '난자 해동 경과일', '배아 이식 경과일', '배아 해동 경과일',
        '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
        '단일 배아 이식 여부', '난자 혼합 경과일',
        '임신 시도 또는 마지막 임신 경과 연수',
    ]
    
    for df in [train_df, test_df]:
        di_mask = df['시술 유형'] == 'DI'
        for col in di_fill_cols:
            if col in df.columns:
                df.loc[di_mask, col] = df.loc[di_mask, col].fillna(0)
    
    knn_cols = [
        '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수',
        '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
        '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
        '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수',
        '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수',
        '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
        '단일 배아 이식 여부',
        '배아 이식 경과일', '난자 혼합 경과일', '배아 해동 경과일',
    ]
    knn_cols = [c for c in knn_cols if c in train_df.columns]
    has_null = [c for c in knn_cols if train_df[c].isnull().any() or test_df[c].isnull().any()]
    
    if has_null:
        print(f"  KNN Imputing {len(has_null)} columns...")
        imputer = KNNImputer(n_neighbors=5, weights='distance')
        train_df[has_null] = imputer.fit_transform(train_df[has_null])
        test_df[has_null] = imputer.transform(test_df[has_null])
        print("  Done.")
    
    return train_df, test_df


# ===================================================================
# 2) Feature Engineering (12번과 동일)
# ===================================================================
def preprocess(df):
    d = df.copy()

    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        if "IUI" in x: return "IUI"
        if "DI" in x:  return "Other"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x: return "IVF"
        return "Other"
    d["시술_대분류"] = d["특정 시술 유형"].apply(major_procedure)

    d["BLASTOCYST_포함"] = d["특정 시술 유형"].astype(str).str.contains("BLASTOCYST", na=False).astype(int)
    d["AH_포함"] = d["특정 시술 유형"].astype(str).str.contains("AH", na=False).astype(int)

    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    d["배아_이식_여부"] = 1 - d[embryo_stage_cols].isna().all(axis=1).astype(int)

    def embryo_stage(row):
        if row["배아_이식_여부"] == 0: return "배아단계_미도달"
        elif pd.isna(row["총 생성 배아 수"]) or row["총 생성 배아 수"] == 0: return "배아생성_실패"
        elif pd.isna(row["이식된 배아 수"]) or row["이식된 배아 수"] == 0: return "이식_미실시"
        else: return "이식_완료"
    d["배아_진행_단계"] = d.apply(embryo_stage, axis=1)

    def collapse_trials(x):
        if x == "0회": return "0회"
        elif x in ["1회","2회"]: return "1–2회"
        else: return "3회 이상"
    d["총시술_bin3"] = d["총 시술 횟수"].apply(collapse_trials)

    def age_group_simple(age):
        if age == "알 수 없음": return "Unknown"
        elif age == "만18-34세": return "34세 이하"
        elif age in ["만35-37세","만38-39세"]: return "35-39세"
        else: return "40세 이상"
    d["나이_3구간"] = d["시술 당시 나이"].apply(age_group_simple)

    def embryo_count_bin(count):
        if pd.isna(count) or count == 0: return "0개"
        elif count <= 2: return "1-2개"
        else: return "3개 이상"
    d["이식배아_구간"] = d["이식된 배아 수"].apply(embryo_count_bin)

    d["Day5_이식_여부"] = (d["배아 이식 경과일"] == 5.0).astype(int)

    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    d["불임_원인_개수"] = d[infertility_cols].sum(axis=1)

    d["배아_해동_실시_여부"] = d["배아 해동 경과일"].notna().astype(int)

    # 비율
    d["배아_이식_비율"] = d["이식된 배아 수"] / (d["총 생성 배아 수"] + 1)
    d["배아_저장_비율"] = d["저장된 배아 수"] / (d["총 생성 배아 수"] + 1)
    d["배아_생성_효율"] = d["총 생성 배아 수"] / (d["수집된 신선 난자 수"] + 1)
    d["미세주입_생성_효율"] = d["미세주입에서 생성된 배아 수"] / (d["미세주입된 난자 수"] + 1)
    d["난자_활용률"] = d["혼합된 난자 수"] / (d["수집된 신선 난자 수"] + 1)

    # 교호작용
    d["나이×Day5"] = d["시술 당시 나이"].astype(str) + "_" + d["Day5_이식_여부"].astype(str)
    d["시술횟수×나이"] = d["총시술_bin3"].astype(str) + "_" + d["나이_3구간"].astype(str)
    d["나이×배아진행"] = d["시술 당시 나이"].astype(str) + "_" + d["배아_진행_단계"]
    d["시기코드×나이"] = d["시술 시기 코드"] + "_" + d["나이_3구간"]
    d["나이×단일이식"] = d["시술 당시 나이"].astype(str) + "_" + d["단일 배아 이식 여부"].fillna(-1).astype(int).astype(str)

    # 배양기간
    d["배양기간"] = d["배아 이식 경과일"] - d["난자 혼합 경과일"]

    # ordinal
    ord_map = {"0회":0, "1회":1, "2회":2, "3회":3, "4회":4, "5회":5, "6회 이상":6}
    for col_name, col_src in [("총시술_ord","총 시술 횟수"), ("IVF시술_ord","IVF 시술 횟수"),
                               ("클리닉시술_ord","클리닉 내 총 시술 횟수"), ("DI시술_ord","DI 시술 횟수"),
                               ("총임신_ord","총 임신 횟수"), ("IVF임신_ord","IVF 임신 횟수"),
                               ("총출산_ord","총 출산 횟수"), ("IVF출산_ord","IVF 출산 횟수"),
                               ("DI임신_ord","DI 임신 횟수"), ("DI출산_ord","DI 출산 횟수")]:
        d[col_name] = d[col_src].map(ord_map)

    # 난자 나이
    def get_egg_age(row):
        if row["난자 출처"] == "본인 제공":
            return row["시술 당시 나이"]
        elif row["난자 출처"] == "기증 제공":
            donor = row["난자 기증자 나이"]
            return donor if donor != "알 수 없음" else "만18-34세"
        return row["시술 당시 나이"]
    d["난자_나이"] = d.apply(get_egg_age, axis=1)

    # 배아 출처
    def get_embryo_source(row):
        fr = 0 if pd.isna(row.get("동결 배아 사용 여부")) else int(row["동결 배아 사용 여부"])
        fs = 0 if pd.isna(row.get("신선 배아 사용 여부")) else int(row["신선 배아 사용 여부"])
        if fr and fs: return "both"
        if fr: return "frozen"
        if fs: return "fresh"
        return "none"
    d["배아_출처"] = d.apply(get_embryo_source, axis=1)

    d["시기×배아출처"] = d["시술 시기 코드"] + "_" + d["배아_출처"]

    # 배아 생성 이유
    def embryo_reason_simple(x):
        if pd.isna(x): return "미도달"
        if "현재 시술용" in x: return "시술용"
        return "저장기증"
    d["배아생성이유"] = d["배아 생성 주요 이유"].apply(embryo_reason_simple)

    # 배란
    d["배란 유도 유형"] = d["배란 유도 유형"].replace({
        '생식선 자극 호르몬': '기록되지 않은 시행', '세트로타이드 (억제제)': '기록되지 않은 시행'
    })
    d["배란_자극_유도"] = d["배란 자극 여부"].astype(str) + "_" + d["배란 유도 유형"].astype(str)

    d["나이_알수없음"] = (d["시술 당시 나이"] == "알 수 없음").astype(int)

    # 불임 남/여 분리
    male_inf = ["불임 원인 - 남성 요인", "불임 원인 - 정자 농도",
                "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태"]
    female_inf = ["불임 원인 - 난관 질환", "불임 원인 - 배란 장애",
                  "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증"]
    d["남성_불임_수"] = d[male_inf].sum(axis=1)
    d["여성_불임_수"] = d[female_inf].sum(axis=1)
    d["여성_불임_비율"] = np.where(
        (d["남성_불임_수"] + d["여성_불임_수"]) == 0, -1,
        d["여성_불임_수"] / (d["남성_불임_수"] + d["여성_불임_수"])
    )

    # 시술 이력 비율
    d["IVF_임신률"] = np.where(d["IVF시술_ord"] == 0, -1, d["IVF임신_ord"] / d["IVF시술_ord"])
    d["IVF_출산률"] = np.where(d["IVF임신_ord"] == 0, -1, d["IVF출산_ord"] / d["IVF임신_ord"])
    d["총_출산률"] = np.where(d["총임신_ord"] == 0, -1, d["총출산_ord"] / d["총임신_ord"])
    d["클리닉_비율"] = np.where(d["총시술_ord"] == 0, -1, d["클리닉시술_ord"] / d["총시술_ord"])

    # 추가 비율
    d["미세주입_이식_비율"] = np.where(d["이식된 배아 수"] == 0, -1, d["미세주입 배아 이식 수"] / d["이식된 배아 수"])
    d["혼합_생성률"] = np.where((d["혼합된 난자 수"] + d["해동 난자 수"]) == 0, -1,
                              d["총 생성 배아 수"] / (d["혼합된 난자 수"] + d["해동 난자 수"]))
    d["미세주입_배아_생성률"] = np.where(d["미세주입된 난자 수"] == 0, -1,
                                     d["미세주입에서 생성된 배아 수"] / d["미세주입된 난자 수"])
    d["총_사용_배아"] = d["해동된 배아 수"] + d["총 생성 배아 수"]
    d["IVF_정자_미혼합"] = ((d["파트너 정자와 혼합된 난자 수"] == 0) &
                          (d["기증자 정자와 혼합된 난자 수"] == 0) &
                          (d["시술 유형"] == "IVF")).astype(int)

    # 시기별 세분화
    d["시기×단일이식"] = d["시술 시기 코드"] + "_" + d["단일 배아 이식 여부"].fillna(0).astype(int).astype(str)
    d["시기×배란자극"] = d["시술 시기 코드"] + "_" + d["배란 자극 여부"].astype(str)
    d["시기×유전진단"] = d["시술 시기 코드"] + "_" + d["착상 전 유전 진단 사용 여부"].fillna(0).astype(int).astype(str)

    # 배양 시간
    d["신선_배양시간"] = (d["배아 이식 경과일"] - d["난자 혼합 경과일"]).fillna(0)
    d["동결_배양시간"] = (d["배아 이식 경과일"] - d["배아 해동 경과일"]).fillna(0)
    d["이상적_배양"] = d["배아 이식 경과일"].isin([3.0, 5.0]).astype(int)

    # 시기별 신선난자/해동배아 구간
    def fresh_egg_tier(row):
        if pd.isna(row["신선 배아 사용 여부"]) or row["신선 배아 사용 여부"] == 0:
            return "not_fresh"
        eggs = row["수집된 신선 난자 수"] if not pd.isna(row["수집된 신선 난자 수"]) else 0
        code = row["시술 시기 코드"]
        if eggs > 10: return f"{code}_high"
        elif eggs > 0: return f"{code}_low"
        return f"{code}_zero"
    d["시기별_신선난자_구간"] = d.apply(fresh_egg_tier, axis=1)

    def frozen_thaw_tier(row):
        if pd.isna(row["동결 배아 사용 여부"]) or row["동결 배아 사용 여부"] == 0:
            return "not_frozen"
        thawed = row["해동된 배아 수"] if not pd.isna(row["해동된 배아 수"]) else 0
        code = row["시술 시기 코드"]
        if thawed > 3: return f"{code}_many"
        elif thawed > 0: return f"{code}_few"
        return f"{code}_zero"
    d["시기별_해동배아_구간"] = d.apply(frozen_thaw_tier, axis=1)

    # PGD/PGS
    d["유전검사_합"] = d["착상 전 유전 진단 사용 여부"].fillna(0) + d["착상 전 유전 검사 사용 여부"].fillna(0)
    d["PGD_실시"] = d["PGD 시술 여부"].notna().astype(int)
    d["PGS_실시"] = d["PGS 시술 여부"].notna().astype(int)
    d["대리모 여부"] = d["대리모 여부"].fillna(-1)

    return d


# ===================================================================
# 3) Main: OOF AutoGluon
# ===================================================================
def main():
    print("=" * 80)
    print("14번: AutoGluon + KNN Imputation + 12번 피처셋")
    print("=" * 80)

    # Load
    train_raw = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
    test_raw  = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
    sub       = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

    # Step 1: Imputation
    print("\n[Step 1] Imputation...")
    train_raw, test_raw = impute_missing(train_raw, test_raw)

    # Step 2: Feature Engineering
    print("\n[Step 2] Feature Engineering...")
    train_df = preprocess(train_raw)
    test_df  = preprocess(test_raw)

    # Drop
    drop_cols = [ID_COL]
    train_df = train_df.drop(columns=[c for c in drop_cols if c in train_df.columns])
    test_df  = test_df.drop(columns=[c for c in drop_cols if c in test_df.columns])

    feature_cols = [c for c in train_df.columns if c != TARGET_COL]
    print(f"Features: {len(feature_cols)}")

    # Step 3: OOF AutoGluon
    print(f"\n[Step 3] AutoGluon OOF ({N_FOLDS}-fold)")
    print(f"  presets={AG_PRESETS}, time_limit={AG_TIME_LIMIT}s/fold")
    print("=" * 60)

    y = train_df[TARGET_COL].values
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    oof_pred = np.zeros(len(train_df))
    test_pred = np.zeros(len(test_df))

    for fold, (tr_idx, va_idx) in enumerate(skf.split(train_df, y), 1):
        print(f"\n--- Fold {fold}/{N_FOLDS} ---")

        tr_data = train_df.iloc[tr_idx].reset_index(drop=True)
        va_data = train_df.iloc[va_idx].reset_index(drop=True)

        # AutoGluon 학습
        ag_path = os.path.join(OUT_DIR, f"ag_fold{fold}")

        predictor = TabularPredictor(
            label=TARGET_COL,
            eval_metric='roc_auc',
            path=ag_path,
        )

        predictor.fit(
            train_data=tr_data,
            time_limit=AG_TIME_LIMIT,
            presets=AG_PRESETS,
            num_stack_levels=AG_NUM_STACK,
            num_bag_folds=AG_NUM_BAG,
            verbosity=2,
        )

        # OOF prediction
        va_pred = predictor.predict_proba(va_data)[1].values
        oof_pred[va_idx] = va_pred

        fold_auc = roc_auc_score(y[va_idx], va_pred)
        print(f"  Fold {fold} AUC: {fold_auc:.6f}")

        # Test prediction
        te_pred = predictor.predict_proba(test_df)[1].values
        test_pred += te_pred / N_FOLDS

        # Leaderboard
        lb = predictor.leaderboard(va_data, silent=True)
        print(f"  Best model: {lb.iloc[0]['model']} (score={lb.iloc[0]['score_val']:.6f})")

        # 메모리 정리
        del predictor
        gc.collect()

    # 결과
    oof_auc = roc_auc_score(y, oof_pred)
    print(f"\n{'='*60}")
    print(f">>> AutoGluon OOF AUC = {oof_auc:.6f}")
    print(f"{'='*60}")

    # Save
    sub2 = sub.copy()
    sub2["probability"] = test_pred
    sub_path = os.path.join(OUT_DIR, f"14_ag_CV{oof_auc:.6f}.csv")
    sub2.to_csv(sub_path, index=False)
    print(f"\nSaved: {sub_path}")

    np.save(os.path.join(OUT_DIR, "14_oof_ag.npy"), oof_pred)
    np.save(os.path.join(OUT_DIR, "14_test_ag.npy"), test_pred)

    with open(os.path.join(OUT_DIR, "14_summary.txt"), "w", encoding="utf-8") as f:
        f.write(f"14번 AutoGluon 파이프라인\n{'='*60}\n")
        f.write(f"OOF AUC: {oof_auc:.6f}\n")
        f.write(f"N_FOLDS: {N_FOLDS}\n")
        f.write(f"AG_TIME_LIMIT: {AG_TIME_LIMIT}s\n")
        f.write(f"AG_PRESETS: {AG_PRESETS}\n")
        f.write(f"Features: {len(feature_cols)}\n")

    print("\nDONE!")


if __name__ == "__main__":
    main()

14번: AutoGluon + KNN Imputation + 12번 피처셋

[Step 1] Imputation...
  KNN Imputing 3 columns...
  Done.

[Step 2] Feature Engineering...
Features: 128

[Step 3] AutoGluon OOF (5-fold)
  presets=best_quality, time_limit=7200s/fold

--- Fold 1/5 ---


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 7200 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/ag_fold1/ds_sub_fit/sub_fit_ho.
2026-02-11 12:28:58,827	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Runn

  Fold 1 AUC: 0.738935
  Best model: WeightedEnsemble_L2 (score=0.741056)

--- Fold 2/5 ---


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 7200 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/ag_fold2/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 1869 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 5331 seconds

  Fold 2 AUC: 0.743414
  Best model: CatBoost_BAG_L1 (score=0.739364)

--- Fold 3/5 ---


...

  Fold 3 AUC: 0.740964
  Best model: WeightedEnsemble_L2 (score=0.740456)

--- Fold 4/5 ---


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 7200 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/ag_fold4/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 1855 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 5345 seconds

  Fold 4 AUC: 0.738970
  Best model: WeightedEnsemble_L2 (score=0.741306)

--- Fold 5/5 ---


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 7200 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/ag_fold5/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 1871 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 5329 seconds

  Fold 5 AUC: 0.741451
  Best model: WeightedEnsemble_L2 (score=0.740358)

>>> AutoGluon OOF AUC = 0.740734

Saved: ../outputs/14_ag_CV0.740734.csv

DONE!
